# Neural Network
Het doel van deze opdracht is om een basisbegrip van neurale netwerken te ontwikkelen door een eenvoudig neuraal netwerk te implementeren, zonder gebruik te maken van backpropagation en gradient descent. Er dient voor het trainen van het model een simpel algoritme gebruikt te worden. Je moet de code zelf goed kan uitleggen.

In [73]:
import pyodbc
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Input

from torch import nn
import torch
from torch.utils.data import DataLoader, TensorDataset

## Database connectie

In [74]:
# database name
DB = {
    'servername': '(local)\\SQLEXPRESS',
    'database': 'DEDS_DataWarehouse'}

In [75]:
export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')


# Create a cursor from the connection
export_cursor = export_conn.cursor()

# check if connection is successful, else throw an error
if export_conn:
    print("Connection with database is established")
else:
    print("Connection with database is not established")
    raise Exception("Connection with database is not established")

Connection with database is established


## Tabellen inlezen
We hebben de volgende tabellen nodig:
- Order_details
- Product
- Order

In [76]:
product_query = "SELECT * FROM Product"

product_result = export_cursor.execute(product_query)
product_fetch = product_result.fetchall()
product_columns = [column[0] for column in product_result.description]
product = pd.DataFrame.from_records(product_fetch, columns=product_columns)

# Dropping current_value and last_updated columns
product = product.drop(columns=['CURRENT_VALUE', 'LAST_UPDATED'])

product

,PRODUCT_SK,PRODUCT_number,PRODUCT_name_product,PRODUCT_description_description,PRODUCT_image_image,PRODUCT_INTRODUCTION_DATE_introduced,PRODUCT_PRODUCTION_COST_cost,PRODUCT_MARGIN_margin,PRODUCT_LANGUAGE_language,PRODUCT_MINIMUM_SALE_PRICE_minPrice,PRODUCT_PRODUCT_LINE_code,PRODUCT_PRODUCT_LINE_code_en,PRODUCT_PRODUCT_TYPE_code,PRODUCT_PRODUCT_TYPE_code_en
0,1,1,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",P01CE1CG1.jpg,2011-02-15,4.0000,0.3300,EN,4.3300,1,Camping Equipment,1,Cooking Gear
1,2,10,TrailChef Utensils,"Spoon, fork and knife set made of a light yet ...",P10CE1CG1.jpg,2011-02-15,10.0000,0.4000,EN,10.4000,1,Camping Equipment,1,Cooking Gear
2,3,100,Insect Bite Relief,The Insect Bite Relief helps the itching and s...,P100OP4FA17.jpg,2011-02-15,3.0000,0.5000,EN,3.5000,4,Outdoor Protection,17,First Aid
3,4,101,Hailstorm Steel Irons,Iron is 17-4 stainless steel. Shafts are grap...,P101GE5IR18.jpg,2019-12-15,305.5400,0.4300,EN,305.9700,5,Golf Equipment,18,Irons
4,5,102,Hailstorm Titanium Irons,Made entirely of pure titanium. The ultimate i...,P102GE5IR18.jpg,2019-10-12,380.9500,0.5100,EN,381.4600,5,Golf Equipment,18,Irons
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,111,95,Sun Shield,"PABA free sunscreen, SPF 30, poison oak and iv...",P91OP4SS16.jpg,2011-02-15,3.0000,0.5000,EN,3.5000,4,Outdoor Protection,16,Sunscreen
111,112,96,Compact Relief Kit,A personal first aid kit is recommended for ev...,P96OP4FA17.jpg,2011-02-15,16.4300,0.2800,EN,16.7100,4,Outdoor Protection,17,First Aid
112,113,97,Deluxe Family Relief Kit,A complete medical kit suitable for families w...,P96OP4FA17.jpg,2013-05-03,25.0000,0.2800,EN,25.2800,4,Outdoor Protection,17,First Aid
113,114,98,Calamine Relief,Use the Calamine Relief for allergic skin reac...,P98OP4FA17.jpg,2011-02-15,3.0000,0.5000,EN,3.5000,4,Outdoor Protection,17,First Aid


In [77]:
order_details_query = "SELECT * FROM Order_details"

order_details_result = export_cursor.execute(order_details_query)
order_details_fetch = order_details_result.fetchall()
order_details_columns = [column[0] for column in order_details_result.description]
order_details = pd.DataFrame.from_records(order_details_fetch, columns=order_details_columns)

# Dropping current_value and last_updated columns
order_details = order_details.drop(columns=['CURRENT_VALUE', 'LAST_UPDATED'])

order_details

,ORDER_DETAILS_SK,ORDER_DETAILS_code,ORDER_DETAILS_QUANTITY_quantity,ORDER_DETAILS_TOTAL_COST_total,ORDER_DETAILS_TOTAL_MARGIN_margin,ORDER_DETAILS_ORDER_NUMBER_order,ORDER_DETAILS_PRODUCT_NUMBER_product,ORDER_DETAILS_UNIT_ID_unit
0,1,100000,16,257.6000,360.6400,4405,112,1
1,2,100001,20,322.0000,450.8000,5008,112,2
2,3,100002,24,386.4000,540.9600,4394,112,3
3,4,100003,18,289.8000,405.7200,4396,112,4
4,5,100004,20,322.0000,450.8000,4382,112,5
...,...,...,...,...,...,...,...,...
43058,43059,99995,146,402.9600,810.3000,4402,111,43059
43059,43060,99996,172,474.7200,954.6000,4400,111,43060
43060,43061,99997,192,529.9200,1065.6000,5148,111,43061
43061,43062,99998,192,529.9200,1065.6000,4384,111,43062


In [78]:
order_query = "SELECT * FROM [Order]"

order_result = export_cursor.execute(order_query)
order_fetch = order_result.fetchall()
order_columns = [column[0] for column in order_result.description]
order = pd.DataFrame.from_records(order_fetch, columns=order_columns)

# Dropping current_value and last_updated columns
order = order.drop(columns=['CURRENT_VALUE', 'LAST_UPDATED'])

order

,ORDER_SK,ORDER_order_number,ORDER_ORDER_METHOD_CODE_method_code,ORDER_ORDER_METHOD_EN_method
0,1,1153,7,Sales visit
1,2,1154,4,E-mail
2,3,1155,7,Sales visit
3,4,1156,5,Web
4,5,1157,2,Telephone
...,...,...,...,...
5355,5356,9475,4,E-mail
5356,5357,9476,5,Web
5357,5358,9477,2,Telephone
5358,5359,9478,7,Sales visit


## Database connectie sluiten

In [79]:
export_cursor.close()
export_conn.close()

## Tabellen samenvoegen

In [80]:
combined_data = pd.merge(order_details, product, left_on='ORDER_DETAILS_PRODUCT_NUMBER_product', right_on='PRODUCT_SK')
combined_data = pd.merge(combined_data, order, left_on='ORDER_DETAILS_ORDER_NUMBER_order', right_on='ORDER_SK')

combined_data

,ORDER_DETAILS_SK,ORDER_DETAILS_code,ORDER_DETAILS_QUANTITY_quantity,ORDER_DETAILS_TOTAL_COST_total,ORDER_DETAILS_TOTAL_MARGIN_margin,ORDER_DETAILS_ORDER_NUMBER_order,ORDER_DETAILS_PRODUCT_NUMBER_product,ORDER_DETAILS_UNIT_ID_unit,PRODUCT_SK,PRODUCT_number,...,PRODUCT_LANGUAGE_language,PRODUCT_MINIMUM_SALE_PRICE_minPrice,PRODUCT_PRODUCT_LINE_code,PRODUCT_PRODUCT_LINE_code_en,PRODUCT_PRODUCT_TYPE_code,PRODUCT_PRODUCT_TYPE_code_en,ORDER_SK,ORDER_order_number,ORDER_ORDER_METHOD_CODE_method_code,ORDER_ORDER_METHOD_EN_method
0,1,100000,16,257.6000,360.6400,4405,112,1,112,96,...,EN,16.7100,4,Outdoor Protection,17,First Aid,4405,8462,5,Web
1,2,100001,20,322.0000,450.8000,5008,112,2,112,96,...,EN,16.7100,4,Outdoor Protection,17,First Aid,5008,9111,2,Telephone
2,3,100002,24,386.4000,540.9600,4394,112,3,112,96,...,EN,16.7100,4,Outdoor Protection,17,First Aid,4394,8451,2,Telephone
3,4,100003,18,289.8000,405.7200,4396,112,4,112,96,...,EN,16.7100,4,Outdoor Protection,17,First Aid,4396,8453,7,Sales visit
4,5,100004,20,322.0000,450.8000,4382,112,5,112,96,...,EN,16.7100,4,Outdoor Protection,17,First Aid,4382,8439,7,Sales visit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43058,43059,99995,146,402.9600,810.3000,4402,111,43059,111,95,...,EN,3.5000,4,Outdoor Protection,16,Sunscreen,4402,8459,4,E-mail
43059,43060,99996,172,474.7200,954.6000,4400,111,43060,111,95,...,EN,3.5000,4,Outdoor Protection,16,Sunscreen,4400,8457,4,E-mail
43060,43061,99997,192,529.9200,1065.6000,5148,111,43061,111,95,...,EN,3.5000,4,Outdoor Protection,16,Sunscreen,5148,9267,5,Web
43061,43062,99998,192,529.9200,1065.6000,4384,111,43062,111,95,...,EN,3.5000,4,Outdoor Protection,16,Sunscreen,4384,8441,4,E-mail


## Tabel opschonen

In [81]:
# dropping columns
keep_columns = ['ORDER_DETAILS_QUANTITY_quantity','PRODUCT_name_product','PRODUCT_PRODUCT_LINE_code', 'PRODUCT_PRODUCT_TYPE_code', 'ORDER_ORDER_METHOD_CODE_method_code']
combined_data = combined_data.loc[:, keep_columns]

combined_data

,ORDER_DETAILS_QUANTITY_quantity,PRODUCT_name_product,PRODUCT_PRODUCT_LINE_code,PRODUCT_PRODUCT_TYPE_code,ORDER_ORDER_METHOD_CODE_method_code
0,16,Compact Relief Kit,4,17,5
1,20,Compact Relief Kit,4,17,2
2,24,Compact Relief Kit,4,17,2
3,18,Compact Relief Kit,4,17,7
4,20,Compact Relief Kit,4,17,7
...,...,...,...,...,...
43058,146,Sun Shield,4,16,4
43059,172,Sun Shield,4,16,4
43060,192,Sun Shield,4,16,5
43061,192,Sun Shield,4,16,4


## Data voorbereiden
data omzetten naar bruikbare data voor het neurale netwerk

In [82]:
# encoding PRODUCT_name_product. Cannot use dummy encoding since there are too many unique values and only one value per row
combined_data['PRODUCT_name_product'] = combined_data['PRODUCT_name_product'].astype('category').cat.codes.astype('int32')

# turning everything into int32
combined_data['PRODUCT_PRODUCT_LINE_code'] = combined_data['PRODUCT_PRODUCT_LINE_code'].astype('int32')
combined_data['ORDER_DETAILS_QUANTITY_quantity'] = combined_data['ORDER_DETAILS_QUANTITY_quantity'].astype('int32')
combined_data['PRODUCT_PRODUCT_TYPE_code'] = combined_data['PRODUCT_PRODUCT_TYPE_code'].astype('int32')
combined_data['ORDER_ORDER_METHOD_CODE_method_code'] = combined_data['ORDER_ORDER_METHOD_CODE_method_code'].astype('int32')

combined_data.dtypes

ORDER_DETAILS_QUANTITY_quantity        int32
PRODUCT_name_product                   int32
PRODUCT_PRODUCT_LINE_code              int32
PRODUCT_PRODUCT_TYPE_code              int32
ORDER_ORDER_METHOD_CODE_method_code    int32
dtype: object

## Data splitsen

In [83]:
# splitting the data in x and y
X = combined_data.drop(columns=['ORDER_DETAILS_QUANTITY_quantity'])
y = combined_data['ORDER_DETAILS_QUANTITY_quantity']

# splitting the data in train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

# printing the sizes
print(f'X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}')

X_train: (34450, 4), X_test: (8613, 4), y_train: (34450,), y_test: (8613,)


In [84]:
# defining the amount of epochs and batch size
epochs = 10
batch_size = 50

## Tensorflow + keras
neural network using keras

### Model maken

In [85]:
# getting the amount of columns in order_details
amount_of_columns = len(combined_data.columns)-1 # needs -1 since one of the columns is the target column. Should be 4

# the model
model = Sequential()

# input layer, 4 nodes
model.add(Input(shape=(amount_of_columns,)))

# 1 hidden layer
model.add(Dense(8, activation='relu'))

# output layer, 1 node
model.add(Dense(1, activation='linear')) # linear since we are predicting a number

# compile the model, so we can get metrics
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error', 'mean_squared_logarithmic_error']) # adam uses gradient descent, doesnt seem to be able to use non-gradient descent based optimizer

### Model trainen

In [86]:
# training the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size) # batch size: after how many samples the weights are updated

Epoch 1/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 2s 621us/step - loss: 5417.0894 - mean_absolute_error: 44.7924 - mean_squared_logarithmic_error: 5.4406
Epoch 2/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 0s 632us/step - loss: 4040.4753 - mean_absolute_error: 36.6848 - mean_squared_logarithmic_error: 1.0667
Epoch 3/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step - loss: 3716.3511 - mean_absolute_error: 36.0835 - mean_squared_logarithmic_error: 1.0449
Epoch 4/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 1s 696us/step - loss: 3987.5698 - mean_absolute_error: 36.9394 - mean_squared_logarithmic_error: 1.0516
Epoch 5/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 1s 874us/step - loss: 3830.5332 - mean_absolute_error: 36.0348 - mean_squared_logarithmic_error: 1.0184
Epoch 6/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 0s 691us/step - loss: 4048.6248 - mean_absolute_error: 35.8609 - mean_squared_logarithmic_error: 1.0045
Epoch 7/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 1s 704us/step - loss: 3808.5090 - mean_absolute_error: 35.9572 - mean_squared_logarithmic_error:

### Model evalueren

In [87]:
# predicting the model
predictions = model.predict(X_test)

# Convert predictions to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['Predicted'], index=X_test.index)

# Add the actual values to the DataFrame
predictions_df['Actual'] = y_test

predictions_df['Difference'] = predictions_df['Predicted'] - predictions_df['Actual']
predictions_df['Absolute Difference'] = abs(predictions_df['Difference'])

predictions_df

270/270 ━━━━━━━━━━━━━━━━━━━━ 0s 692us/step


,Predicted,Actual,Difference,Absolute Difference
9130,55.161983,56,-0.838017,0.838017
27718,48.000538,36,12.000538,12.000538
41898,45.553394,26,19.553394,19.553394
15673,35.316696,24,11.316696,11.316696
2314,32.487080,46,-13.512920,13.512920
...,...,...,...,...
38076,48.037392,46,2.037392,2.037392
18731,52.282925,94,-41.717075,41.717075
2460,45.536724,56,-10.463276,10.463276
15719,32.151806,90,-57.848194,57.848194


In [88]:
# evaluating the model
loss, mae, mslr = model.evaluate(X_test, y_test)
average_difference = predictions_df['Absolute Difference'].mean()
print(f'Loss (Mean Squared Error): {loss}, Mean Absolute Error: {mae}, Mean Squared Logarithmic Error: {mslr}, Average Difference: {average_difference}')

270/270 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step - loss: 2895.9229 - mean_absolute_error: 33.6853 - mean_squared_logarithmic_error: 0.9131
Loss (Mean Squared Error): 3389.097412109375, Mean Absolute Error: 34.24584197998047, Mean Squared Logarithmic Error: 0.9257012009620667, Average Difference: 34.245828257168434


## Pytorch
neural network using pytorch

### Model maken

In [89]:
# Define the architecture of the model
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 16), # input layer with 16 output nodes
    nn.ReLU(),
    nn.Linear(16, 1) # 16 input nodes to 1 output node
)

### Model trainen

In [90]:
# Custom optimizer class which doesn't use gradient descent or backpropagation
class SimpleErrorOptimizerPyTorch(torch.optim.Optimizer):
    def __init__(self, params, lr=0.01):
        defaults = dict(lr=lr)
        super(SimpleErrorOptimizerPyTorch, self).__init__(params, defaults)

    def step(self, closure=None):
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                # Here we directly subtract the error from the parameters
                p.data -= group['lr'] * p.grad.data

In [91]:
# Converting the dataframes to tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Define the loss function and the custom optimizer
criterion = nn.MSELoss()
optimizer = SimpleErrorOptimizerPyTorch(model.parameters())

# Define the DataLoader
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

# Train the model
for epoch in range(epochs):  # 100 epochs
    for inputs, targets in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

Epoch 1/10, Loss: 2319.421875
Epoch 2/10, Loss: 4029.596923828125
Epoch 3/10, Loss: 3339.003662109375
Epoch 4/10, Loss: 3947.5771484375
Epoch 5/10, Loss: 1041.998779296875
Epoch 6/10, Loss: 4434.98291015625
Epoch 7/10, Loss: 3079.8955078125
Epoch 8/10, Loss: 1479.3380126953125
Epoch 9/10, Loss: 1294.7119140625
Epoch 10/10, Loss: 1157.0465087890625


### Model evalueren

In [92]:
# Evaluate the model
with torch.no_grad():
    predictions = model(X_test_tensor).squeeze()
    loss = criterion(predictions, y_test_tensor)
print(f'Test Loss: {loss.item()}')

Test Loss: 3317.273193359375


## Scratch
Een neural network maken zonder gebruik te maken van een library

### Model maken

In [93]:
class CustomNeuralNetwork:
    def __init__(self, input_nodes, hidden_nodes, output_nodes):
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        # Initialize weights and biases
        self.weights_input_to_hidden = np.random.normal(0.0, self.input_nodes**-0.5,
                                       (self.input_nodes, self.hidden_nodes))

        self.weights_hidden_to_output = np.random.normal(0.0, self.hidden_nodes**-0.5,
                                       (self.hidden_nodes, self.output_nodes))
        self.bias_hidden = np.zeros(self.hidden_nodes)
        self.bias_output = np.zeros(self.output_nodes)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def forward_pass(self, X):
        hidden_layer_input = np.dot(X, self.weights_input_to_hidden) + self.bias_hidden
        hidden_layer_output = self.sigmoid(hidden_layer_input)

        output_layer_input = np.dot(hidden_layer_output, self.weights_hidden_to_output) + self.bias_output
        output_layer_output = self.sigmoid(output_layer_input)

        return hidden_layer_output, output_layer_output

    def calculate_error(self, y, final_outputs):
        return y - final_outputs

    def update_weights(self, X, error, lr):
        # Simple rule for updating weights
        self.weights_input_to_hidden += lr * np.dot(X.T, error).mean(axis=1)[:, None]
        self.weights_hidden_to_output += lr * np.dot(X.T, error).mean(axis=1)[:, None]

    def train(self, X, y, lr):
        _, final_outputs = self.forward_pass(X)
        error = self.calculate_error(y, final_outputs)
        self.update_weights(X, error, lr)

    def predict(self, X):
        _, final_outputs = self.forward_pass(X)
        return final_outputs

In [94]:
# Initialize the neural network
nn = CustomNeuralNetwork(X_train.shape[1], 8, 1)

### Model trainen

In [96]:
# Train the model
for epoch in range(epochs):
    nn.train(X_train.values, y_train.values, 0.01)
    
    _, final_outputs = nn.forward_pass(X_train.values)
    loss = np.mean(np.square(y_train.values - final_outputs))
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss}')

ValueError: operands could not be broadcast together with shapes (4,8) (34450,1) (4,8) 

### Model evalueren

In [ ]:
# Generate predictions for the test data
predictions = nn.predict(X_test.values)

# Calculate the error
error = y_test.values - predictions

# Square the error
squared_error = np.square(error)

# Calculate the mean of the squared errors
mse = np.mean(squared_error)

print(f'Mean Squared Error: {mse}')

predictions